In [1]:
#Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr
import os; import time
import pickle
import h5py

In [2]:
#MAIN DIRECTORIES
mainDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
mainCodeDirectory=os.path.join(mainDirectory,"Code/CodeFiles/")
scratchDirectory='/homez/air673/koa_scratch/'
codeDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/CodeFiles/Project_Algorithms/Domain_Profiles/'

In [3]:
#LOADING DATA
def GetDataDirectories(simulationNumber):
    if simulationNumber == 1:
        Directory=os.path.join(mainDirectory,'Model/cm1r20.3/run')
        res='1km'; t_res='5min'; Np_str='1e6'; Nz_str='34'
    elif simulationNumber == 2:
        Directory=scratchDirectory
        res='1km'; t_res='1min'; Np_str='50e6'; Nz_str='95'
    elif simulationNumber == 3:
        Directory=scratchDirectory
        res='250m'; t_res='1min'; Np_str='50e6'; Nz_str='95'
        
    dataDirectory = os.path.join(Directory, f"cm1out_{res}_{t_res}_{Nz_str}nz.nc")
    parcelDirectory = os.path.join(Directory,f"cm1out_pdata_{res}_{t_res}_{Np_str}np.nc")
    return dataDirectory, parcelDirectory, res,t_res,Np_str,Nz_str
    
def GetData(dataDirectory, parcelDirectory):
    dataNC = xr.open_dataset(dataDirectory, decode_timedelta=True) 
    parcelNC = xr.open_dataset(parcelDirectory, decode_timedelta=True) 
    return dataNC,parcelNC

def SubsetDataVars(dataNC):
    varList = ["thflux", "qvflux", "tsk", "cape", 
               "cin", "lcl", "lfc", "th",
               "prs", "rho", "qv", "qc",
               "qr", "qi", "qs","qg", 
               "buoyancy", "uinterp", "vinterp", "winterp",]
    
    varList += ["ptb_hadv", "ptb_vadv", "ptb_hidiff", "ptb_vidiff",
                "ptb_hturb", "ptb_vturb", "ptb_mp", "ptb_rdamp", 
                "ptb_rad", "ptb_div", "ptb_diss",]
    
    varList += ["qvb_hadv", "qvb_vadv", "qvb_hidiff", "qvb_vidiff", 
                "qvb_hturb", "qvb_vturb", "qvb_mp",]
    
    varList += ["wb_hadv", "wb_vadv", "wb_hidiff", "wb_vidiff",
                "wb_hturb", "wb_vturb", "wb_pgrad", "wb_rdamp", "wb_buoy",]

    return dataNC[varList]

[dataDirectory,parcelDirectory, res,t_res,Np_str,Nz_str] = GetDataDirectories(simulationNumber=1)
[data,parcel] = GetData(dataDirectory, parcelDirectory)

In [ ]:
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'

In [ ]:
import sys
path=os.path.join(mainCodeDirectory,'Functions/')
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions

# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

In [ ]:
def load_vars(data):
    # print('PRESSURE VARIABLES'); ################################# PRESSURE VARIABLES
    p0=1e5
    P=data['prs'].data

   # print('THERMODYNAMICS'); ################################# THERMODYNAMICS
    Rd=287.04 
    # Rv=461.5
    cpd=1005.7 #+-2.5 #****divide by this
    cpv=1870 #+-25
    cpl=4190 #+-30 #Cpl = Cvl = Cl
    cpi=2106.0 #Cpi = Cvi = Ci
    Lv0=2.501e6
    Lf0=0.3337e6

    def Lv(T): #Kirchoff's formula L_i,ii= L_i,ii0+(Cpii-Cpi)*(T-273.15)
        Llv=Lv0+(cpv-cpl)*(T-273.15)
        return Llv
        
    def Lf(T): #Kirchoff's formula L_i,ii= L_i,ii0+(Cpii-Cpi)*(T-273.15)
        Lli=Lf0+(cpi-cpl)*(T-273.15) 
        return Lli
    
    
    # print('TEMPERATURE'); ################################# TEMPERATURE
    theta=data['th'].data
    T=theta*(P/p0)**(Rd/cpd)

    # print('Specific Humidity'); ################################# Specific Humidity
    rv=data['qv'].data
    rl=data['qr'].data+data['qc'].data
    rt=rv+rl

    ri=data['qi'].data
    rT=rt+ri

    ################################# Geopotential Height
    Nz,Ny,Nx=len(data['zh']),len(data['yh']),len(data['xh'])
    zh_values=data['zh'].values*1000
    Z=np.broadcast_to(zh_values[:, np.newaxis, np.newaxis], (Nz, Ny, Nx))
    g=9.81
    gZ=g*Z
    
    return cpd,cpl,T, Lv,Lf, rv,ri,rt,rT, gZ

In [ ]:
def make_MSE(cpd,cpl,T,Lv,Lf,rv,ri,rt,rT,gZ):
    # MSE=(cpd+rt*cpl)*T + (Lv(T)*rv) + ( (1+rt)*(gZ) )
    MSE=(cpd+rT*cpl)*T + (Lv(T)*rv) - (Lf(T)*ri) + ( (1+rT)*(gZ) ) #including ice
    return MSE

In [ ]:
if res=='1km':
    outputDirectory=os.path.join(mainDirectory,'Code','OUTPUT','Variable_Calculation','CalculateMoreVariables')
    os.makedirs(outputDirectory, exist_ok=True)
if res=='250m':
    outputDirectory=os.path.join(scratchDirectory,'OUTPUT','Variable_Calculation','CalculateMoreVariables')
    os.makedirs(outputDirectory, exist_ok=True)
    
def initiate_array():
    # Define array dimensions (adjust based on your data)
    t_size = len(data['time'])  # Number of timesteps
    z_size = len(data['zh'])    # Number of vertical levels
    y_size = len(data['yh'])    # Number of y-axis points
    x_size = len(data['xh'])    # Number of x-axis points

    out_file=os.path.join(outputDirectory, 'MSE'+f'_{res}_{t_res}'+'.h5')
    with h5py.File(out_file, 'a') as f:
        # Check if the dataset 'theta_e' already exists
        if 'MSE' not in f:
            # Create a dataset with the full size for all time steps (initially empty)
            f.create_dataset('MSE', 
                             (t_size, z_size, y_size, x_size),  # Full size for all timesteps
                             maxshape=(None, z_size, y_size, x_size),  # Unlimited timesteps (can grow along time dimension)
                             dtype='float64', 
                             chunks=(1, z_size, y_size, x_size))  # Chunks for time axis to allow resizing

            
def add_timestep_at_index(timestep_data, index):
    out_file=os.path.join(outputDirectory, 'MSE'+f'_{res}_{t_res}'+'.h5')
    with h5py.File(out_file, 'a') as f:
        # Access the existing dataset 'MSE'
        dataset = f['MSE']
        
        # Assign the new timestep data at the specified index
        dataset[index] = timestep_data

In [ ]:
#RUNNING

In [ ]:
#MAKING ARRAY TO STORE THETA_E
initiate_array()

#CALCULATING AND APPENDING TO DATA EACH TIMESTEP
for t in range(len(data['time'])):
    if np.mod(t,1)==0: print(f'Current time {t}')
    data_t=data.isel(time=t)
    [cpd,cpl,T,Lv,Lf,rv,ri,rt,rT,gZ] = load_vars(data_t)
    MSE=make_MSE(cpd,cpl,T,Lv,Lf,rv,ri,rt,rT,gZ)
    add_timestep_at_index(MSE, t)

#Fast for 1KM, longer for 250m

In [ ]:
# #READING FINAL OUTPUT
# dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
# # dir2='/mnt/lustre/koa/scratch/air673/'
# with h5py.File(dir2 + 'Variable_Calculation/' + 'MSE'+f'_{res}_{t_res}'+'.h5', 'a') as f:
#     # Access the existing dataset 'MSE'
#     dataset = f['MSE'][:]

In [ ]:
# plt.contourf(dataset[50,0])
# plt.colorbar(label='J/kg')
# plt.title("MSE at t = 50, z = 30 m")